It is highly recommended to use a powerful **GPU**, you can use it for free uploading this notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb).
<table align="center">
 <td align="center"><a target="_blank" href="https://colab.research.google.com/github/ezponda/intro_deep_learning/blob/main/class/NLP/semantic_search_QA_clustering.ipynb">
        <img src="https://colab.research.google.com/img/colab_favicon_256px.png"  width="50" height="50" style="padding-bottom:5px;" />Run in Google Colab</a></td>
  <td align="center"><a target="_blank" href="https://github.com/ezponda/intro_deep_learning/blob/main/class/NLP/semantic_search_QA_clustering.ipynb">
        <img src="https://github.githubassets.com/images/modules/logos_page/GitHub-Mark.png"  width="50" height="50" style="padding-bottom:5px;" />View Source on GitHub</a></td>
</table>

# Semantic search & QA

In this notebook, we'll introduce semantic search and question-answering using [`sentence-transformers`](https://www.sbert.net/), a Python library for state-of-the-art sentence, text and image embeddings. These embeddings are useful for semantic similarity tasks, such as information retrieval and question-answering systems.

#embedding

把一句话“编码成向量嵌入”，可以这样理解：

直觉版

把文本 → 数字坐标：模型把一句话映射到一个高维坐标点，比如 384 维或 768 维的向量 v。

语义近的＝坐标近：意思相近的句子落在空间里彼此更近；不相关的落得远。

用相似度（常用余弦相似度）量化“近不近”。

为什么要这样做

计算机不懂词义，但会算向量。有了向量，就能做：

语义检索（找最相似的段落）

去重/聚类（找相似问题）

语义匹配、推荐、排序

怎么得到这个向量

用预训练模型（如 sentence-transformers 的 bi-encoder）。

训练目标常见是对比学习：让“正样本对”（同义/配对的句子）更近，“负样本对”更远。

推理时：emb = model.encode(text) → 得到一个定长向量。

向量里的每一维表示什么？

它们是分布式特征：不是“第 1 维=情感，第 2 维=地点”这种可读标签，而是模型自动学到的抽象语义轴。单维很难解释，但整体方向可用来比相似度。

相似度怎么量

常用余弦相似度（比较方向，不看长度）：

cos
(
𝑢
,
𝑣
)
=
𝑢
⋅
𝑣
∥
𝑢
∥
∥
𝑣
∥
cos(u,v)=
∥u∥∥v∥
u⋅v
	​


实践里常把向量归一化（单位长度）后做检索，更稳。

一个玩具例子（想象 3 维）

“I love cats” →
𝑣
1
=
(
0.6
,

0.7
,

0.2
)
v
1
	​

=(0.6,0.7,0.2)

“I like kittens” →
𝑣
2
=
(
0.58
,

0.72
,

0.18
)
v
2
	​

=(0.58,0.72,0.18) （与
𝑣
1
v
1
	​

 很近 → 语义相似）

“The stock market fell” →
𝑣
3
=
(
−
0.1
,

−
0.3
,

0.9
)
v
3
	​

=(−0.1,−0.3,0.9) （方向差很多 → 不相似）

和“三维/高维”关系

这是一个高维空间（几十到上千维）。维度=定位一个点所需的最少数字个数。这里的点=一句话的语义位置。

好的实践

选择合适的模型尺寸（如 all-MiniLM-L6-v2 384 维，速度快；更大模型更准但慢）。

批量文本先编码并缓存/索引（如 FAISS/Annoy/HNSW），查询时再计算相似度。

需要更准的最终排序时，用 CrossEncoder 对 top-k 重新打分。

In [ ]:
# Install the sentence-transformers library
!pip install -U sentence-transformers

In [3]:
import json
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import time
import gzip
import os

一句话总结：Embedding = 把语义装进向量，让“意义的远近”变成“几何的远近”。

We'll use a pre-trained Sentence Transformer model to generate sentence embeddings. Many pre-trained models are available [here](https://www.sbert.net/docs/pretrained_models.html)

In [ ]:
model_name = 'all-MiniLM-L6-v2'
model = SentenceTransformer(model_name)

For our semantic search and question-answering task, we need a list of documents or paragraphs to search through for relevant information.

In [5]:
# Sample paragraphs
paragraphs = [
    "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.",
    "The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor within New York City, in the United States.",
    "The Great Wall of China is a series of fortifications made of stone, brick, tamped earth, wood, and other materials, generally built along an east-to-west line across the historical northern borders of China.",
    "The Colosseum, also known as the Flavian Amphitheatre, is an oval amphitheatre in the centre of the city of Rome, Italy.",
    "The Taj Mahal is an ivory-white marble mausoleum on the southern bank of the river Yamuna in the Indian city of Agra."
]

paragraphs = np.array(paragraphs)

In [6]:
# Generate embeddings for paragraphs
corpus_embeddings = model.encode(paragraphs)
print(corpus_embeddings.shape)

(5, 384)


这两格代码在做一件事：把 5 段示例文本编码成向量嵌入，为后续语义检索做准备。

paragraphs = [...]：准备 5 段关于地标的英文段落（语料库 corpus）。

paragraphs = np.array(paragraphs)：转成 numpy 数组，便于批量编码。

corpus_embeddings = model.encode(paragraphs)：用你上面加载的 SentenceTransformer 模型，把每段文本 → 一个定长向量。

print(corpus_embeddings.shape) 输出 (5, 384)：表示共有 5 个段落，每个段落被编码成 384 维向量（你用的模型的默认维度）。

这些嵌入稍后会用余弦相似度与查询向量比对，找出语义最相近的段落。

Now, let's define a function to perform semantic search, given a query and a list of paragraph embeddings.

In [7]:
def semantic_search(query, model, corpus_embeddings, paragraphs, top_k=2):
    query_embedding = model.encode([query])[0] # # 1) 把查询句子编码成向量
    similarities = cosine_similarity([query_embedding], corpus_embeddings)[0] # # 2) 计算查询与库中每个段落的余弦相似度（返回形状 1×N，取第0行变成(N,)）
    indexes = np.argpartition(similarities, -top_k)[-top_k:] #3) 先用 argpartition 取出相似度前 top_k 名（O(N) 近似线性，快）
    indexes = indexes[np.argsort(-similarities[indexes])] # 4) 再在这 top_k 里按分数从大到小精确排序
    print(f"Input query: {query}")
    print()
    for text, sim in zip(list(paragraphs[indexes]), similarities[indexes].tolist()):
        print(f"{sim:.3f}\t{text}")


semantic_search('Where is the Colosseum', model, corpus_embeddings, paragraphs, top_k=2)

Input query: Where is the Colosseum

0.801	The Colosseum, also known as the Flavian Amphitheatre, is an oval amphitheatre in the centre of the city of Rome, Italy.
0.226	The Taj Mahal is an ivory-white marble mausoleum on the southern bank of the river Yamuna in the Indian city of Agra.


## Multilingual models


In [8]:
# lets try in other languages
semantic_search('¿Dónde está el Coliseo?', model, corpus_embeddings, paragraphs, top_k=2)

Input query: ¿Dónde está el Coliseo?

0.086	The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor within New York City, in the United States.
0.067	The Taj Mahal is an ivory-white marble mausoleum on the southern bank of the river Yamuna in the Indian city of Agra.


We have multilinguals models available [here](https://www.sbert.net/docs/pretrained_models.html#multi-lingual-models)

In [ ]:
# we can use multilingual models
model_name = 'paraphrase-multilingual-MiniLM-L12-v2'
multi_model = SentenceTransformer(model_name)

In [10]:
multi_corpus_embeddings = multi_model.encode(paragraphs)
print(multi_corpus_embeddings.shape)

(5, 384)


In [11]:
semantic_search('¿Dónde está el Coliseo?', multi_model, multi_corpus_embeddings, paragraphs, top_k=2)

Input query: ¿Dónde está el Coliseo?

0.439	The Colosseum, also known as the Flavian Amphitheatre, is an oval amphitheatre in the centre of the city of Rome, Italy.
0.299	The Statue of Liberty is a colossal neoclassical sculpture on Liberty Island in New York Harbor within New York City, in the United States.


## Wikipedia semantic search

As dataset, we use Simple English Wikipedia. Compared to the full English wikipedia, it has only
about 170k articles. We split these articles into paragraphs

In [ ]:
wikipedia_filepath = 'data/simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    util.http_get('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        for paragraph in data['paragraphs']:
            # We encode the passages as [title, text]
            passages.append(data['title']+':  '+ paragraph)

# If you like, you can also limit the number of passages you want to use
print("Passages:", len(passages))
print(passages[0])
print(passages[1])

In [13]:
reduced_passages = np.array(passages[:5000])
reduced_passages.shape

(5000,)

In [ ]:
corpus_embeddings = model.encode(reduced_passages, show_progress_bar=True)

In [15]:
semantic_search('Best american actor', model, corpus_embeddings, reduced_passages, top_k=2)

Input query: Best american actor

0.539	Aaron Kwok:  Aaron won the Best Actor Award again at the forty-third Golden Horse Awards on 24 November 2006 for his role in the movie "After This Our Exile". He became the second actor in the history of the Golden Horse Awards to win the Best Actor Award year after year. Jackie Chan first achieved this back in the 1990s.
0.425	James L. Brooks:  He is best known for creating American television programs such as "The Mary Tyler Moore Show", "The Simpsons", "Rhoda" and "Taxi". His best-known movie is "Terms of Endearment", for which he received three Academy Awards in 1984.


In [16]:
semantic_search('Number countries Europe', model, corpus_embeddings, reduced_passages, top_k=2)

Input query: Number countries Europe

0.502	European Union member state:  A European Union member state is any one of the twenty-seven countries that have joined the European Union (EU) since it was found in 1958 as the European Economic Community (EEC). From an original membership of six states, there have been five successive enlargements. The largest happened on 1 May 2004, when ten member states joined.
0.465	European Space Agency:  The member countries of ESA are Austria, Belgium, Czech Republic, Denmark, Finland, France, Germany, Greece, Ireland, Italy, Luxembourg, the Netherlands, Norway, Portugal, Spain, Sweden, Switzerland and the United Kingdom.


### Question1: Load a different pre-trained Sentence Transformer model and compare its performance to the last model on the same set of paragraphs and queries. Which model performs better?

In [ ]:
# Load a different pre-trained model, generate embeddings, and test with the same queries
model_name = 'all-mpnet-base-v2'
new_model = SentenceTransformer(model_name)

## Question 2: Find text duplicates

Try to find duplicate or near-duplicate texts in a given corpus based on their semantic similarity using sentence-transformers.

In [18]:
corpus = [
    "The quick brown fox jumps over the lazy dog.",
    "The quick brown fox leaps over the lazy dog.",
    "The sky is blue, and the grass is green.",
    "The grass is green, and the sky is blue.",
    "It's a sunny day today.",
    "The weather is sunny today.",
    "She was wearing a beautiful red dress.",
    "She had on a gorgeous red dress.",
    "I'm going to the supermarket to buy some groceries.",
    "I'm heading to the supermarket to purchase some groceries.",
    "He didn't like the movie because it was too long.",
    "He disliked the movie as it was too lengthy.",
    "The train was delayed due to technical issues.",
    "Technical issues caused the train to be delayed.",
    "I'll have a cup of coffee with milk and sugar, please.",
    "Can I get a coffee with milk and sugar, please?",
    "The conference was very informative and interesting.",
    "The conference turned out to be interesting and informative.",
    "He enjoys listening to classical music in his free time.",
    "In his leisure time, he likes to listen to classical music.",
    "Please make sure you turn off the lights before leaving.",
    "Before leaving, ensure that you switch off the lights.",
    "The boy was delighted with the gift he received.",
    "Receiving the present made the young lad ecstatic.",
    "She has a preference for Italian cuisine.",
    "Her favorite type of food is from Italy.",
    "The software engineer resolved the issue by modifying the code.",
    "By altering the programming, the tech expert fixed the problem.",
    "Due to the inclement weather, the baseball game was postponed.",
    "The baseball match was rescheduled because of bad weather conditions.",
    "The house was engulfed in a raging fire.",
    "Flames rapidly consumed the residence.",
    "He is constantly browsing the internet for the latest news.",
    "He frequently scours the web to stay updated on current events.",
    "The puppy was playing with a toy in the garden.",
    "In the yard, the young dog was frolicking with its plaything.",
    "The artist painted a beautiful landscape on the canvas.",
]

In [19]:
# Step 1: Initialize the SentenceTransformer model
model = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
# Step 2: Obtain corpus embeddings
embeddings = model.encode(corpus, show_progress_bar=True)

In [21]:
# Step 3: Calculate similarity and find duplicates

# TODO: Define a similarity threshold
similarity_threshold = 0.85 #设定阈值。余弦相似度范围是 [-1, 1]（一般语义嵌入落在 [0, 1]）。分数高于 0.85 就认为“很像/可能重复”。

# TODO: Iterate over each pair of embeddings in the corpus
# Calculate the cosine similarity between the embeddings
# If the similarity is above the threshold, add the sentences to the duplicates list
duplicates = []

for i, emb1 in enumerate(embeddings): #外层循环：遍历第 i 个向量 emb1。embeddings 是你之前算好的语料库嵌入矩阵，形如 (N, D)。
    for j, emb2 in enumerate(embeddings[i + 1:]): #内层循环：只和后面的向量比较（从 i+1 开始）。
        similarity = cosine_similarity([emb1], [emb2])[0][0] #计算 余弦相似度。
        if similarity > similarity_threshold:
            duplicates.append((corpus[i], corpus[i + j + 1], similarity))

In [22]:
print("Duplicate sentences:")
for sent1, sent2, sim in duplicates:
    print(f"{sent1} | {sent2} | Similarity: {sim:.2f}")
    print()

Duplicate sentences:
The quick brown fox jumps over the lazy dog. | The quick brown fox leaps over the lazy dog. | Similarity: 0.99

The sky is blue, and the grass is green. | The grass is green, and the sky is blue. | Similarity: 0.98

It's a sunny day today. | The weather is sunny today. | Similarity: 0.93

She was wearing a beautiful red dress. | She had on a gorgeous red dress. | Similarity: 0.98

I'm going to the supermarket to buy some groceries. | I'm heading to the supermarket to purchase some groceries. | Similarity: 0.98

He didn't like the movie because it was too long. | He disliked the movie as it was too lengthy. | Similarity: 0.94

The train was delayed due to technical issues. | Technical issues caused the train to be delayed. | Similarity: 0.98

I'll have a cup of coffee with milk and sugar, please. | Can I get a coffee with milk and sugar, please? | Similarity: 0.85

The conference was very informative and interesting. | The conference turned out to be interesting and

# Document Clustering

K-means clustering is a popular unsupervised machine learning algorithm that groups data points into k clusters based on their similarity. In our case, we want to group documents based on their semantic similarity. The algorithm requires us to specify the number of clusters k in advance.

K-means 聚类是常见的无监督机器学习算法，它根据相似度把数据点分成 k 个簇。
在这里，我们想根据文档的语义相似性来分组。
该算法需要我们事先指定簇的数量 k。

用 K-means 给文档分群。

核心思路

先把每个文档编码成向量嵌入（sentence-transformers）。

在向量空间里，相似文档距离更近。

用 K-means 把这些点分成 K 组；每组用一个质心代表主题。

做语义聚类时，常用余弦相似度。做法是：把嵌入单位化（L2=1），这样用 K-means 的欧氏距离≈最大化余弦相似度。

选择 K（多少簇）

Elbow（手肘法）：看 SSE 随 K 的下降拐点。

Silhouette：选平均轮廓系数最高的 K。

如果不想先定 K，试 HDBSCAN（密度聚类，能丢噪声点）。

In [23]:
corpus = [
    "The apple is a sweet fruit",
    "Oranges are citrus fruits",
    "Bananas are rich in potassium",
    "Strawberries are red fruits",
    "Dogs are domesticated animals",
    "Cats are also pets",
    "Elephants are the largest land mammals",
    "Cows provide us with milk",
    "Sharks are marine predators",
    "Whales are the largest marine mammals",
    "Dolphins are very intelligent",
    "Artificial intelligence is the future",
    "Machine learning is a subset of AI",
    "Deep learning is a part of machine learning",
    "Neural networks are used in deep learning",
]

df = pd.DataFrame({'documents': corpus})

In [24]:
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode the documents in the corpus
document_embeddings = model.encode(corpus)

In [25]:
from sklearn.cluster import KMeans
num_clusters = 3
clustering_model = KMeans(n_clusters=num_clusters, init='k-means++', max_iter=300, n_init=10)
clustering_model.fit(document_embeddings)
cluster_assignment = clustering_model.labels_

df['cluster'] = cluster_assignment

In [26]:
for i in range(num_clusters):
    print(f"Cluster {i}:")
    print(df[df['cluster'] == i]['documents'].values, "\n")

Cluster 0:
['The apple is a sweet fruit' 'Oranges are citrus fruits'
 'Bananas are rich in potassium' 'Strawberries are red fruits'] 

Cluster 1:
['Dogs are domesticated animals' 'Cats are also pets'
 'Elephants are the largest land mammals' 'Cows provide us with milk'
 'Sharks are marine predators' 'Whales are the largest marine mammals'
 'Dolphins are very intelligent'] 

Cluster 2:
['Artificial intelligence is the future'
 'Machine learning is a subset of AI'
 'Deep learning is a part of machine learning'
 'Neural networks are used in deep learning'] 



# Community Detection with Sentence Transformers

The sentence_transformers library provides a utility for community detection which applies a threshold on the cosine similarity score to identify distinct communities of sentences that are semantically similar. This method can be particularly helpful for organizing a large corpus of text into meaningful groups.




The [`community_detection`](https://www.sbert.net/docs/package_reference/util.html#sentence_transformers.util.community_detection) function in the Sentence Transformers library is a useful utility for finding clusters or communities of semantically similar sentences. Here are the details of the function parameters:

- `document_embeddings`: This is the list of embeddings for the documents in your corpus. The embeddings can be created using any of the Sentence Transformer models. The embeddings should be in the form of a 2D tensor or a list of 1D tensors. Each embedding should be a fixed-length vector that represents the semantic meaning of a document.

- `threshold`: This is a float value between 0 and 1 that determines the cutoff for considering two documents to be part of the same community. It's based on the cosine similarity of the document embeddings. If the cosine similarity of two document embeddings is greater than the threshold, those two documents are considered to be in the same community. The higher the threshold, the more similar the documents in each community will be. However, a higher threshold may also result in more communities.

- `min_community_size`: This is the minimum number of documents that a community must have. If a community has fewer than this number of documents, it will be discarded. The default value is 1, but you might want to set a higher value if you're interested in larger communities. This can help filter out noise and find more meaningful communities.

- `batch_size`: As the function computes cosine similarities between document pairs, it may consume a significant amount of memory for a large corpus. To manage this, the computations are done in batches. The batch_size parameter determines the number of document pairs to compute similarities for in each batch. Larger batch sizes can be faster but consume more memory, while smaller batch sizes are slower but more memory-efficient.

The function returns a list of communities, where each community is a list of indices in the original list of documents. Each community represents a group of semantically similar documents based on the provided threshold.

用 Sentence Transformers 做社群/社区检测


sentence_transformers 提供了一个“社区检测”工具：基于余弦相似度阈值，把语义相近的句子划分到不同的“社区”。这在整理大规模文本语料时很有用。
community_detection 函数可用于寻找语义相似句子的簇/社区。

参数说明：


document_embeddings：语料中各文档/句子的嵌入列表。可用任意 Sentence Transformer 模型得到。应为 2D 张量或 1D 张量列表；每个嵌入是表示语义的定长向量。


threshold：取值 0~1，用来判定两条文档是否应归为同一社区，依据是它们嵌入的余弦相似度。相似度高于该阈值 → 视为同一社区。阈值越高，社区内部越相似，但社区数量可能变多。


min_community_size：社区的最小文档数。小于该数量的社区会被丢弃。默认 1；若想得到更有意义的社区，可设大一些以过滤噪声。


batch_size：计算两两余弦相似度会占用较多内存，因此按批计算。batch_size 控制每批计算的文档对数量。批越大更快但更耗内存；批小更省内存但更慢。


返回值：一个社区列表；每个社区是原始文档列表中的索引集合，表示在给定阈值下语义相似的一组文档。

In [ ]:
from sentence_transformers.util import community_detection
document_embeddings = model.encode(
    corpus, show_progress_bar=True, convert_to_tensor=True
)
communities = community_detection(
    document_embeddings, threshold=0.5, min_community_size=2, batch_size=1024
)
for i, comm in enumerate(communities):
    print('_'*50)
    print(f'community: {i}, size: {len(comm)}')
    print('\n'.join([corpus[ind] for ind in comm]))
    print()

In the output, we will see the communities of semantically similar sentences. Note that the choice of the threshold value can greatly affect the results: a lower threshold will result in larger but less cohesive communities, while a higher threshold will result in smaller but more tightly-knit communities.

The community_detection function is a fast and efficient way to group similar sentences together, but keep in mind that it's a rather simple method based on thresholding the cosine similarity, and more sophisticated community detection methods might yield better results for certain tasks or datasets.

This function is a great way to explore the semantic structure of your corpus and to get a high-level understanding of the main themes or topics in your text data.

在输出中，我们会看到语义相似句子形成的社区。注意：阈值的选择会明显影响结果——阈值低，会得到更大但内聚性较弱的社区；阈值高，会得到更小但更紧密的社区。

community_detection 函数能快速高效地把相似句子分到一起，但要记住它只是基于余弦相似度阈值的简单方法；在某些任务或数据集上，更复杂的社区检测方法可能效果更好。

这个函数很适合探索语料的语义结构，帮助你从高层次理解文本数据里的主要主题或话题。